In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load pre-trained model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Encode input text
input_text = "Today the whether is"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Check if pad_token_id is None and handle it
pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
attention_mask = input_ids.ne(pad_token_id).float()

# Generate text with adjusted parameters
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=1000,
    num_return_sequences=1,
    do_sample=True,           # Enable sampling
    top_k=50,                 # Top-k sampling
    top_p=0.95,               # Top-p sampling
    temperature=0.7,          # Temperature
    repetition_penalty=1.2,   # Penalty for repetition
    pad_token_id=pad_token_id
)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Today the whether is a big one.
I'm in this for now, and I need to see it right before my face."
